Import


In [3]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt


création de la dataframe

In [4]:
df = pd.read_csv(r"After_Sort.csv", parse_dates=["BaseDateTime"])
df.sort_values("BaseDateTime", inplace=True)

print informations

In [5]:
print(df.head())
print(df.info())
print(df.describe())
print(df["VesselType"].value_counts())

             id       MMSI BaseDateTime       LAT       LON   SOG    COG  \
117352  9030257  563020500   2023-05-25  28.82655 -89.43528  14.2  207.9   
117395  9032593  366954420   2023-05-25  27.77639 -82.63030   0.1  294.9   
117339  9029199  215583000   2023-05-25  26.48967 -79.39617  10.2  182.7   
117360  9030460  367181550   2023-05-25  29.85450 -89.98148   0.0  298.2   
117442  9036191  368045410   2023-05-25  27.84124 -97.06945   5.9   15.5   

        Heading        VesselName            IMO CallSign  VesselType  Status  \
117352    206.0       MERCURY SKY     IMO9796949   9V5148          70       0   
117395    294.0  PROVINCETOWN III     IMO9329394  WDB8185          60       0   
117339    184.0         COBIA LNG     IMO9869306  9HA5192          84       0   
117360      NaN   BELLE CHASSE II            NaN  WDD7069          60       0   
117442      NaN    JOSEPH F WEBER  IMO1012865430  WDK3453          60       0   

        Length  Width  Draft  Cargo TransceiverClass  
1

ajout des colones lat et long +5, +10, +15 et delta_temps à la dataframe

In [6]:
nouvelles_colonnes = ['LONG_5', 'LAT_5', 'LONG_10', 'LAT_10', 'LONG_15', 'LAT_15','Delta_temps']

# Ajout des colonnes avec des valeurs vides (NaN)
for col in nouvelles_colonnes:
    df[col] = None  # ou pd.NA, ou np.nan si vous utilisez numpy


print(df.head())

             id       MMSI BaseDateTime       LAT       LON   SOG    COG  \
117352  9030257  563020500   2023-05-25  28.82655 -89.43528  14.2  207.9   
117395  9032593  366954420   2023-05-25  27.77639 -82.63030   0.1  294.9   
117339  9029199  215583000   2023-05-25  26.48967 -79.39617  10.2  182.7   
117360  9030460  367181550   2023-05-25  29.85450 -89.98148   0.0  298.2   
117442  9036191  368045410   2023-05-25  27.84124 -97.06945   5.9   15.5   

        Heading        VesselName            IMO  ... Draft  Cargo  \
117352    206.0       MERCURY SKY     IMO9796949  ...   NaN   70.0   
117395    294.0  PROVINCETOWN III     IMO9329394  ...   1.9   60.0   
117339    184.0         COBIA LNG     IMO9869306  ...   9.4   84.0   
117360      NaN   BELLE CHASSE II            NaN  ...   NaN   60.0   
117442      NaN    JOSEPH F WEBER  IMO1012865430  ...   3.3   60.0   

        TransceiverClass  LONG_5  LAT_5  LONG_10  LAT_10 LONG_15 LAT_15  \
117352                 A    None   None     Non

Remplissage de la colonne Delta_temps

In [7]:
df['Delta_temps'] = df.sort_values(['MMSI', 'BaseDateTime']).groupby('MMSI')['BaseDateTime'].diff().dt.total_seconds()

OOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO

In [8]:
from datetime import timedelta

def remplir_positions(df):
    df_result = []

    # Trier les données
    df = df.sort_values(["MMSI", "BaseDateTime"]).reset_index(drop=True)

    for mmsi, group in df.groupby("MMSI"):
        group = group.sort_values("BaseDateTime").reset_index(drop=True)
        for idx, row in group.iterrows():
            base_time = row["BaseDateTime"]

            # Fenêtres de temps en minutes
            t_windows = {
                '5': (base_time + timedelta(minutes=3), base_time + timedelta(minutes=7)),
                '10': (base_time + timedelta(minutes=8), base_time + timedelta(minutes=12)),
                '15': (base_time + timedelta(minutes=13), base_time + timedelta(minutes=17)),
            }

            positions = {}
            for key, (start, end) in t_windows.items():
                candidates = group[(group["BaseDateTime"] >= start) & (group["BaseDateTime"] <= end)]
                if not candidates.empty:
                    chosen = candidates.iloc[0]
                    positions[f"LAT_{key}"] = chosen["LAT"]
                    positions[f"LONG_{key}"] = chosen["LON"]
                else:
                    break  # Si une fenêtre n’a pas de donnée, on saute cette ligne
            else:
                df_result.append({**row, **positions})

    return pd.DataFrame(df_result)

# Appliquer sur la dataframe complète avant split
df_cleaned = remplir_positions(df)

# Affichage pour vérification
print(df_cleaned.head())
print(f"Nombre de lignes après nettoyage : {len(df_cleaned)}")

        id       MMSI        BaseDateTime       LAT       LON  SOG    COG  \
0  9625845  205776000 2023-05-25 00:31:27  25.95848 -97.37879  0.0   77.2   
1  9314767  205776000 2023-05-25 00:55:27  25.95842 -97.37880  0.0  162.9   
2  9353676  205776000 2023-05-25 01:04:27  25.95846 -97.37879  0.1  169.9   
3  9376837  205776000 2023-05-25 01:07:27  25.95845 -97.37879  0.2  185.8   
4  9422207  205776000 2023-05-25 01:13:27  25.95843 -97.37879  0.0  172.0   

   Heading VesselName         IMO  ... Draft  Cargo  TransceiverClass  \
0    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
1    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
2    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
3    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
4    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   

     LONG_5     LAT_5   LONG_10    LAT_10   LONG_15    LAT_15  Delta_temps  
0 -97.37882  25.95846

In [9]:
# Récupérer les MMSI uniques de df_cleaned
mmsi_uniques = df_cleaned['MMSI'].dropna().unique()

# Mélanger
np.random.seed(42)
np.random.shuffle(mmsi_uniques)

# Répartition MMSI
n_train = 97
n_test = 30
n_val = 22

mmsi_train = mmsi_uniques[:n_train]
mmsi_test = mmsi_uniques[n_train:n_train + n_test]
mmsi_val = mmsi_uniques[n_train + n_test:n_train + n_test + n_val]

# Création des sous-ensembles
df_train = df_cleaned[df_cleaned['MMSI'].isin(mmsi_train)]
df_test = df_cleaned[df_cleaned['MMSI'].isin(mmsi_test)]
df_val = df_cleaned[df_cleaned['MMSI'].isin(mmsi_val)]

# Vérification
print("Nouveaux ensembles à partir de df_cleaned :")
print("Train:", df_train['MMSI'].nunique())
print("Test :", df_test['MMSI'].nunique())
print("Val  :", df_val['MMSI'].nunique())


Nouveaux ensembles à partir de df_cleaned :
Train: 97
Test : 30
Val  : 22


Séparation en bases d'apprentissage, test et validation par MMSI

Observation de début de chaque df

In [10]:
print(df_train.head(5))
print("\n")
print("\n")
print("  ===============================================================  ")
print("\n")
print("\n")
print(df_test.head(5))
print("\n")
print("\n")
print("  ===============================================================  ")
print("\n")
print("\n")
print(df_val.head(5))

        id       MMSI        BaseDateTime       LAT       LON  SOG    COG  \
0  9625845  205776000 2023-05-25 00:31:27  25.95848 -97.37879  0.0   77.2   
1  9314767  205776000 2023-05-25 00:55:27  25.95842 -97.37880  0.0  162.9   
2  9353676  205776000 2023-05-25 01:04:27  25.95846 -97.37879  0.1  169.9   
3  9376837  205776000 2023-05-25 01:07:27  25.95845 -97.37879  0.2  185.8   
4  9422207  205776000 2023-05-25 01:13:27  25.95843 -97.37879  0.0  172.0   

   Heading VesselName         IMO  ... Draft  Cargo  TransceiverClass  \
0    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
1    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
2    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
3    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
4    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   

     LONG_5     LAT_5   LONG_10    LAT_10   LONG_15    LAT_15  Delta_temps  
0 -97.37882  25.95846

In [11]:
df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)
print(df_train.head(10))


        id       MMSI        BaseDateTime       LAT       LON  SOG    COG  \
0  9625845  205776000 2023-05-25 00:31:27  25.95848 -97.37879  0.0   77.2   
1  9314767  205776000 2023-05-25 00:55:27  25.95842 -97.37880  0.0  162.9   
2  9353676  205776000 2023-05-25 01:04:27  25.95846 -97.37879  0.1  169.9   
3  9376837  205776000 2023-05-25 01:07:27  25.95845 -97.37879  0.2  185.8   
4  9422207  205776000 2023-05-25 01:13:27  25.95843 -97.37879  0.0  172.0   
5  9456484  205776000 2023-05-25 01:22:27  25.95847 -97.37880  0.1  151.4   
6  9477296  205776000 2023-05-25 01:28:27  25.95845 -97.37885  0.1  176.9   
7  9538373  205776000 2023-05-25 01:31:27  25.95841 -97.37876  0.1  158.2   
8  9524383  205776000 2023-05-25 01:37:27  25.95844 -97.37877  0.1  161.9   
9  9570341  205776000 2023-05-25 01:43:27  25.95845 -97.37878  0.1  160.8   

   Heading VesselName         IMO  ... Draft  Cargo  TransceiverClass  \
0    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
1    2

C:\Users\Paul\AppData\Local\Temp\ipykernel_20320\703263359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)


In [12]:
print(f"Taille de l'ensemble d'apprentissage : {len(df_train)}")
print(f"Taille de l'ensemble de test : {len(df_test)}")
print(f"Taille de l'ensemble de validation : {len(df_val)}")


Taille de l'ensemble d'apprentissage : 255966
Taille de l'ensemble de test : 73807
Taille de l'ensemble de validation : 53845


In [13]:
df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)
print(df_train.head())
# print(df_train.iloc[2950:3000])


        id       MMSI        BaseDateTime       LAT       LON  SOG    COG  \
0  9625845  205776000 2023-05-25 00:31:27  25.95848 -97.37879  0.0   77.2   
1  9314767  205776000 2023-05-25 00:55:27  25.95842 -97.37880  0.0  162.9   
2  9353676  205776000 2023-05-25 01:04:27  25.95846 -97.37879  0.1  169.9   
3  9376837  205776000 2023-05-25 01:07:27  25.95845 -97.37879  0.2  185.8   
4  9422207  205776000 2023-05-25 01:13:27  25.95843 -97.37879  0.0  172.0   

   Heading VesselName         IMO  ... Draft  Cargo  TransceiverClass  \
0    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
1    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
2    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
3    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   
4    249.0    LA LUNA  IMO9846079  ...  11.8   70.0                 A   

     LONG_5     LAT_5   LONG_10    LAT_10   LONG_15    LAT_15  Delta_temps  
0 -97.37882  25.95846

C:\Users\Paul\AppData\Local\Temp\ipykernel_20320\1592785229.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.sort_values(["MMSI","BaseDateTime"], inplace=True)


In [37]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

features = ['SOG', 'COG', 'Heading', 'LAT', 'LON']
X = df_train[features]
y_lat = df_train['LAT_5']
y_lon = df_train['LONG_5']

model_lat = RandomForestRegressor(
    n_estimators=20,
    min_samples_leaf=5,
    min_samples_split=5
)
model_lat.fit(X, y_lat)

model_lon = RandomForestRegressor(
    n_estimators=20,
    min_samples_leaf=5,
    min_samples_split=5
)
model_lon.fit(X, y_lon)


,n_estimators,20
,criterion,'squared_error'
,max_depth,None
,min_samples_split,5
,min_samples_leaf,5
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [40]:
import joblib

# Sauvegarde
joblib.dump(model_lat, 'model5_lat.pkl')
joblib.dump(model_lon, 'model5_lon.pkl')

# model_lat = joblib.load('model_lat.pkl')
# model_lon = joblib.load('model_lon.pkl')

['model5_lon.pkl']

In [41]:
from sklearn.metrics import mean_squared_error
import numpy as np

X_test = df_test[features]
y_lat_test = df_test['LAT_5']
y_lon_test = df_test['LONG_5']

lat_pred = model_lat.predict(X_test)
lon_pred = model_lon.predict(X_test)

rmse_lat = np.sqrt(mean_squared_error(y_lat_test, lat_pred))
rmse_lon = np.sqrt(mean_squared_error(y_lon_test, lon_pred))
print("RMSE LAT:", rmse_lat)
print("RMSE LON:", rmse_lon)


RMSE LAT: 0.0059733609094987416
RMSE LON: 0.005980037174169128


In [42]:
from sklearn.metrics import mean_squared_error
import numpy as np

X_test = df_val[features]
y_lat_test = df_val['LAT_5']
y_lon_test = df_val['LONG_5']

lat_pred = model_lat.predict(X_test)
lon_pred = model_lon.predict(X_test)

rmse_lat = np.sqrt(mean_squared_error(y_lat_test, lat_pred))
rmse_lon = np.sqrt(mean_squared_error(y_lon_test, lon_pred))
print("RMSE LAT:", rmse_lat)
print("RMSE LON:", rmse_lon)


RMSE LAT: 0.020294903204915345
RMSE LON: 0.005859311641606743


A

In [43]:
features = ['SOG', 'COG', 'Heading','LAT', 'LON']
X = df_train[features]
y_lat = df_train['LAT_10']
y_lon = df_train['LONG_10']

model_lat10 = RandomForestRegressor(
    n_estimators=20,
    min_samples_leaf=5,
    min_samples_split=5)
model_lat10.fit(X, y_lat)

model_lon10 = RandomForestRegressor(
    n_estimators=20,
    min_samples_leaf=5,
    min_samples_split=5)
model_lon10.fit(X, y_lon)

,n_estimators,20
,criterion,'squared_error'
,max_depth,None
,min_samples_split,5
,min_samples_leaf,5
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [44]:
import joblib

# Sauvegarde
joblib.dump(model_lat, 'model10_lat.pkl')
joblib.dump(model_lon, 'model10_lon.pkl')

# model_lat = joblib.load('model_lat.pkl')
# model_lon = joblib.load('model_lon.pkl')

['model10_lon.pkl']

In [45]:
from sklearn.metrics import mean_squared_error
import numpy as np

X_test = df_test[features]
y_lat_test = df_test['LAT_10']
y_lon_test = df_test['LONG_10']

lat_pred = model_lat.predict(X_test)
lon_pred = model_lon.predict(X_test)

rmse_lat = np.sqrt(mean_squared_error(y_lat_test, lat_pred))
rmse_lon = np.sqrt(mean_squared_error(y_lon_test, lon_pred))
print("RMSE LAT:", rmse_lat)
print("RMSE LON:", rmse_lon)


RMSE LAT: 0.009512296081226704
RMSE LON: 0.010458919390690904


In [46]:
X_test = df_val[features]
y_lat_test = df_val['LAT_10']
y_lon_test = df_val['LONG_10']

lat_pred = model_lat.predict(X_test)
lon_pred = model_lon.predict(X_test)

rmse_lat = np.sqrt(mean_squared_error(y_lat_test, lat_pred))
rmse_lon = np.sqrt(mean_squared_error(y_lon_test, lon_pred))
print("RMSE LAT:", rmse_lat)
print("RMSE LON:", rmse_lon)


RMSE LAT: 0.022397173695154527
RMSE LON: 0.012789891177647537


In [47]:
features = ['SOG', 'COG', 'Heading', 'LAT', 'LON']
X = df_train[features]
y_lat = df_train['LAT_15']
y_lon = df_train['LONG_15']

model_lat15 = RandomForestRegressor(
    n_estimators=20,
    min_samples_leaf=5,
    min_samples_split=5
)
model_lat15.fit(X, y_lat)

model_lon15 = RandomForestRegressor(   
    n_estimators=20,
    min_samples_leaf=5,
    min_samples_split=5)
model_lon15.fit(X, y_lon)

,n_estimators,20
,criterion,'squared_error'
,max_depth,None
,min_samples_split,5
,min_samples_leaf,5
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [48]:
# Sauvegarde
joblib.dump(model_lat, 'model15_lat.pkl')
joblib.dump(model_lon, 'model15_lon.pkl')

# model_lat = joblib.load('model_lat.pkl')
# model_lon = joblib.load('model_lon.pkl')

['model15_lon.pkl']

In [49]:
X_test = df_test[features]
y_lat_test = df_test['LAT_15']
y_lon_test = df_test['LONG_15']

lat_pred = model_lat.predict(X_test)
lon_pred = model_lon.predict(X_test)

rmse_lat = np.sqrt(mean_squared_error(y_lat_test, lat_pred))
rmse_lon = np.sqrt(mean_squared_error(y_lon_test, lon_pred))
print("RMSE LAT:", rmse_lat)
print("RMSE LON:", rmse_lon)


RMSE LAT: 0.014662946624735535
RMSE LON: 0.016501355128220298


In [25]:
X_test = df_val[features]
y_lat_test = df_val['LAT_15']
y_lon_test = df_val['LONG_15']

lat_pred = model_lat.predict(X_test)
lon_pred = model_lon.predict(X_test)

rmse_lat = np.sqrt(mean_squared_error(y_lat_test, lat_pred))
rmse_lon = np.sqrt(mean_squared_error(y_lon_test, lon_pred))
print("RMSE LAT:", rmse_lat)
print("RMSE LON:", rmse_lon)


RMSE LAT: 0.02556332255630216
RMSE LON: 0.02049407517519165


prendre entre 295 et 305sec
prendre entre 595 et 605sec
prendre entre 895 et 905sec

In [26]:
def interpoler_positions_dataframe(df):
    """
    Fonction pour interpoler les positions à 5, 10 et 15 minutes
    pour un dataframe donné
    """
    # Créer une copie pour éviter les warnings
    df_copy = df.copy()
    
    # Trier par MMSI et datetime pour assurer l'ordre chronologique
    df_copy = df_copy.sort_values(['MMSI', 'BaseDateTime']).reset_index(drop=True)
    
    print(f"Début de l'interpolation pour {len(df_copy)} lignes...")
    
    # Parcourir chaque navire
    for mmsi in df_copy['MMSI'].unique():
        # Filtrer les données du navire actuel
        mask_navire = df_copy['MMSI'] == mmsi
        indices_navire = df_copy[mask_navire].index.tolist()
        
        # Parcourir les positions de ce navire
        for i in range(len(indices_navire) - 1):
            idx_actuel = indices_navire[i]
            idx_suivant = indices_navire[i + 1]
            
            # Récupérer les positions et temps
            lat_A = df_copy.loc[idx_actuel, 'LAT']
            lon_A = df_copy.loc[idx_actuel, 'LON']
            time_A = df_copy.loc[idx_actuel, 'BaseDateTime']
            
            lat_B = df_copy.loc[idx_suivant, 'LAT']
            lon_B = df_copy.loc[idx_suivant, 'LON']
            time_B = df_copy.loc[idx_suivant, 'BaseDateTime']
            
            # Calculer l'intervalle de temps en secondes
            delta_temps = (time_B - time_A).total_seconds()
            
            # Interpolation à 5 minutes (300s)
            if delta_temps >= 300:
                fraction_5min = 300 / delta_temps
                lat_5min = lat_A + fraction_5min * (lat_B - lat_A)
                lon_5min = lon_A + fraction_5min * (lon_B - lon_A)
                
                df_copy.loc[idx_actuel, 'LAT_5'] = lat_5min
                df_copy.loc[idx_actuel, 'LONG_5'] = lon_5min
            
            # Interpolation à 10 minutes (600s)
            if delta_temps >= 600:
                fraction_10min = 600 / delta_temps
                lat_10min = lat_A + fraction_10min * (lat_B - lat_A)
                lon_10min = lon_A + fraction_10min * (lon_B - lon_A)
                
                df_copy.loc[idx_actuel, 'LAT_10'] = lat_10min
                df_copy.loc[idx_actuel, 'LONG_10'] = lon_10min
            
            # Interpolation à 15 minutes (900s)
            if delta_temps >= 900:
                fraction_15min = 900 / delta_temps
                lat_15min = lat_A + fraction_15min * (lat_B - lat_A)
                lon_15min = lon_A + fraction_15min * (lon_B - lon_A)
                
                df_copy.loc[idx_actuel, 'LAT_15'] = lat_15min
                df_copy.loc[idx_actuel, 'LONG_15'] = lon_15min
    
    print("Interpolation terminée !")
    return df_copy

# Application de l'interpolation sur les trois ensembles
print("=== INTERPOLATION DE L'ENSEMBLE D'APPRENTISSAGE ===")
df_train_interpolated = interpoler_positions_dataframe(df_train)

print("\n=== INTERPOLATION DE L'ENSEMBLE DE TEST ===")
df_test_interpolated = interpoler_positions_dataframe(df_test)

print("\n=== INTERPOLATION DE L'ENSEMBLE DE VALIDATION ===")
df_val_interpolated = interpoler_positions_dataframe(df_val)

=== INTERPOLATION DE L'ENSEMBLE D'APPRENTISSAGE ===
Début de l'interpolation pour 255966 lignes...
Interpolation terminée !

=== INTERPOLATION DE L'ENSEMBLE DE TEST ===
Début de l'interpolation pour 73807 lignes...
Interpolation terminée !

=== INTERPOLATION DE L'ENSEMBLE DE VALIDATION ===
Début de l'interpolation pour 53845 lignes...
Interpolation terminée !


In [27]:
def verifier_interpolation(df, nom_ensemble):
    """
    Fonction pour vérifier les résultats de l'interpolation
    """
    print(f"\n=== VÉRIFICATION - {nom_ensemble} ===")
    
    # Compter les valeurs non nulles pour chaque colonne d'interpolation
    interpolations_5min = df['LAT_5'].notna().sum()
    interpolations_10min = df['LAT_10'].notna().sum()
    interpolations_15min = df['LAT_15'].notna().sum()
    
    print(f"Interpolations à 5 minutes : {interpolations_5min}")
    print(f"Interpolations à 10 minutes : {interpolations_10min}")
    print(f"Interpolations à 15 minutes : {interpolations_15min}")
    
    # Afficher quelques exemples d'interpolation
    exemples = df[df['LAT_5'].notna()].head(3)
    if len(exemples) > 0:
        print(f"\nExemples d'interpolation à 5 minutes :")
        for idx, row in exemples.iterrows():
            print(f"MMSI {row['MMSI']} : Position originale ({row['LAT']:.5f}, {row['LON']:.5f}) -> Position à +5min ({row['LAT_5']:.5f}, {row['LONG_5']:.5f})")

# Vérification des trois ensembles
verifier_interpolation(df_train_interpolated, "TRAIN")
verifier_interpolation(df_test_interpolated, "TEST")
verifier_interpolation(df_val_interpolated, "VALIDATION")

print("\n=== INTERPOLATION TERMINÉE POUR TOUS LES ENSEMBLES ===")
print("Les DataFrames df_train_interpolated, df_test_interpolated et df_val_interpolated")
print("contiennent maintenant les positions interpolées dans les colonnes LAT_5, LONG_5, LAT_10, LONG_10, LAT_15, LONG_15")


=== VÉRIFICATION - TRAIN ===
Interpolations à 5 minutes : 255966
Interpolations à 10 minutes : 255966
Interpolations à 15 minutes : 255966

Exemples d'interpolation à 5 minutes :
MMSI 205776000 : Position originale (25.95848, -97.37879) -> Position à +5min (25.95847, -97.37879)
MMSI 205776000 : Position originale (25.95842, -97.37880) -> Position à +5min (25.95844, -97.37879)
MMSI 205776000 : Position originale (25.95846, -97.37879) -> Position à +5min (25.95845, -97.37879)

=== VÉRIFICATION - TEST ===
Interpolations à 5 minutes : 73807
Interpolations à 10 minutes : 73807
Interpolations à 15 minutes : 73807

Exemples d'interpolation à 5 minutes :
MMSI 212040000 : Position originale (29.54265, -89.76524) -> Position à +5min (29.54266, -89.76525)
MMSI 212040000 : Position originale (29.54266, -89.76523) -> Position à +5min (29.54266, -89.76525)
MMSI 212040000 : Position originale (29.54266, -89.76525) -> Position à +5min (29.54264, -89.76525)

=== VÉRIFICATION - VALIDATION ===
Interpola

In [28]:
seuil_SOG = 0.5  # à ajuster selon ton dataset
df_filtre = df[df['SOG'] > seuil_SOG].reset_index(drop=True)
len(df_filtre)

132876